In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets.widgets import Layout, HBox, VBox
import bqplot as bqp 
import bqplot.pyplot as plt
import numpy as np

In [2]:
data_df = pd.read_csv('data.csv')
temp_df = data_df[~data_df["YearFrom"].isin(["16..","17.."])]  
temp_df = temp_df.astype({"YearFrom": float})
# temp_df["YearFrom"] = pd.to_numeric(temp_df["YearFrom"])

In [3]:
# pd.options.display.max_columns = None
# temp_df

In [4]:
years=widgets.IntRangeSlider(
    value=[1500, 1803],
    min=1500, max=1803, step=1,
    description='Select Year:',
    layout=Layout(width='1000px'),
    continuous_update=False)

In [5]:
crime_df =temp_df["Misdrijven/Crimes"].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))].value_counts().rename_axis('unique_values').reset_index(name='counts')
crime_df = crime_df.sort_values('counts', ascending=False).head(5)
punish_df =temp_df['punishment(fand)/Punishmandts/Pandalties'].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))].value_counts().rename_axis('unique_values').reset_index(name='counts')
punish_df = punish_df.sort_values('counts', ascending=False).head(5)

filter_df = temp_df[["Misdrijven/Crimes", 'punishment(fand)/Punishmandts/Pandalties', 'VerdachteGeslacht/AccusedGandder','Age', "Suspect Professional/AccusedProfession"]].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))]
filter_df.index = pd.RangeIndex(len(filter_df.index))

trend_df = temp_df["YearFrom"].value_counts().rename_axis('unique_values').reset_index(name='counts')

gender_df = filter_df["VerdachteGeslacht/AccusedGandder"].value_counts().rename_axis('unique_values').reset_index(name='counts')

profession_df = filter_df["Suspect Professional/AccusedProfession"].value_counts().rename_axis('unique_values').reset_index(name='counts')
profession_df = profession_df.sort_values('counts', ascending=False).head(5)

clicked_age_index = []
clicked_gender = None

In [10]:
trend_x_sc = bqp.LinearScale()
trend_y_sc = bqp.LinearScale()

trend_x_ax = bqp.Axis(
    label='Year', 
    scale=trend_x_sc, 
    grid_lines='solid', 
#     tick_format='0f'
)

trend_y_ax = bqp.Axis(
    label='Number of Crimes', 
    scale=trend_y_sc, 
    orientation='vertical', 
#     tick_format='0f'
)

trend = bqp.Scatter(x=trend_df['unique_values'], y=trend_df['counts'], scales={'x': trend_x_sc, 'y': trend_y_sc} , colors=['blue'],display_legend = False, labels=['y1'],stroke_width = 1.0) 
trend_fig = bqp.Figure(
    layout=Layout(width='1000px', border ='solid 1px gray'),
    axes=[trend_x_ax, trend_y_ax], 
    marks=[trend], 
    fig_margin=dict(left=100, top=50, bottom=50, right=100))


gender_bar_fig = plt.figure()
gender_bar = plt.barh(x=[unique_value for unique_value in gender_df['unique_values']],y=[count for count in gender_df['counts']]
, interactions={'click': 'select'}, select_bars=True) 


hist_fig = plt.figure()
hist = plt.hist(filter_df['Age'], interactions={'click': 'select'}, select_bars=True)

profession_bar_fig = plt.figure()
profession_bar = plt.barh(x=[unique_value for unique_value in profession_df['unique_values']],y=[count for count in profession_df['counts']]
, interactions={'click': 'select'}, select_bars=True) 

crime_x_ord = bqp.OrdinalScale()
crime_y_sc = bqp.LinearScale()

punish_x_ord = bqp.OrdinalScale()
punish_y_sc = bqp.LinearScale()

crime_bar = bqp.Bars(
    x = crime_df['unique_values'],
    y = crime_df['counts'],
    scales={'x': crime_x_ord, 'y': crime_y_sc}, 
    orientation='horizontal'
)


crime_x_ax = bqp.Axis(
    label='Frequency',
    scale=crime_y_sc
)

crime_y_ax = bqp.Axis(
#     label='',
    scale=crime_x_ord,
    orientation='vertical'
)

punish_x_ax = bqp.Axis(
    label='Frequency',
    scale=punish_y_sc
)

punish_y_ax = bqp.Axis(
#     label='',
    scale=punish_x_ord,
    orientation='vertical'
)

crime_fig = bqp.Figure(
    layout=Layout(width='1920px', height=str(len(crime_df)*12+90)+'px', border ='solid 1px gray'),
    axes=[crime_x_ax, crime_y_ax],
    marks=[crime_bar],
    fig_margin=dict(top=10, bottom=80, left=835, right=20),
    animation_duration=500
)

punish_bar = bqp.Bars(
    x = punish_df['unique_values'],
    y = punish_df['counts'],
    scales={'x': punish_x_ord, 'y': punish_y_sc}, 
    orientation='horizontal'
)

punish_fig = bqp.Figure(
    layout=Layout(width='1920px', height=str(len(punish_df)*12+90)+'px', border ='solid 1px gray'),
    axes=[punish_x_ax, punish_y_ax],
    marks=[punish_bar],
    fig_margin=dict(top=10, bottom=80, left=835, right=20),
    animation_duration=500
)

SyntaxError: invalid syntax (<ipython-input-10-d0903be61f1c>, line 11)

In [7]:
def click_hist(clicked):
    global clicked_age_index, clicked_gender
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        clicked_gender = None
        clicked_age_index=clicked['new']
        crime_df= filter_df[filter_df.index.isin(clicked['new'])].iloc[:,0].value_counts().rename_axis('unique_values').reset_index(name='counts')
        crime_df = crime_df.sort_values('counts', ascending=False).head(5)
        punish_df= filter_df[filter_df.index.isin(clicked['new'])].iloc[:,1].value_counts().rename_axis('unique_values').reset_index(name='counts') 
        punish_df = punish_df.sort_values('counts', ascending=False).head(5)
        gender_df = filter_df[filter_df.index.isin(clicked['new'])]["VerdachteGeslacht/AccusedGandder"].value_counts().rename_axis('unique_values').reset_index(name='counts')
        profession_df = filter_df[filter_df.index.isin(clicked['new'])]["Suspect Professional/AccusedProfession"].value_counts().rename_axis('unique_values').reset_index(name='counts')
        profession_df = profession_df.sort_values('counts', ascending=False).head(5)
        crime_bar.x = crime_df['unique_values']
        crime_bar.y = crime_df['counts']
        crime_fig.layout.height = str(len(crime_df)*12+90)+'px'
        punish_bar.x = punish_df['unique_values']
        punish_bar.y = punish_df['counts']
        punish_fig.layout.height = str(len(punish_df)*12+90)+'px'
        gender_bar.x=[unique_value for unique_value in gender_df['unique_values']]
        gender_bar.y=[count for count in gender_df['counts']]
        profession_bar.x=[unique_value for unique_value in profession_df['unique_values']]
        profession_bar.y=[count for count in profession_df['counts']]

def click_gender_bar(clicked):
    global clicked_gender 
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        clicked_gender = gender_df['unique_values'][clicked['new'][0]]
        if clicked_age_index == []:            
            crime_df= filter_df.loc[filter_df["VerdachteGeslacht/AccusedGandder"]==gender_df['unique_values'][clicked['new'][0]]].iloc[:,0].value_counts().rename_axis('unique_values').reset_index(name='counts')
            punish_df= filter_df[filter_df["VerdachteGeslacht/AccusedGandder"]==gender_df['unique_values'][clicked['new'][0]]].iloc[:,1].value_counts().rename_axis('unique_values').reset_index(name='counts') 
            profession_df= filter_df[filter_df["VerdachteGeslacht/AccusedGandder"]==gender_df['unique_values'][clicked['new'][0]]]["Suspect Professional/AccusedProfession"].value_counts().rename_axis('unique_values').reset_index(name='counts') 
        else:
            crime_df= filter_df[filter_df.index.isin(clicked_age_index)].loc[filter_df["VerdachteGeslacht/AccusedGandder"]==gender_df['unique_values'][clicked['new'][0]]].iloc[:,0].value_counts().rename_axis('unique_values').reset_index(name='counts')  
            punish_df= filter_df[filter_df.index.isin(clicked_age_index)].loc[filter_df["VerdachteGeslacht/AccusedGandder"]==gender_df['unique_values'][clicked['new'][0]]].iloc[:,1].value_counts().rename_axis('unique_values').reset_index(name='counts') 
            profession_df= filter_df[filter_df.index.isin(clicked_age_index)].loc[filter_df["VerdachteGeslacht/AccusedGandder"]==gender_df['unique_values'][clicked['new'][0]]]["Suspect Professional/AccusedProfession"].value_counts().rename_axis('unique_values').reset_index(name='counts') 
        crime_df = crime_df.sort_values('counts', ascending=False).head(5)
        punish_df = punish_df.sort_values('counts', ascending=False).head(5)
        profession_df = profession_df.sort_values('counts', ascending=False).head(5)
        crime_bar.x = crime_df['unique_values']
        crime_bar.y = crime_df['counts']
        crime_fig.layout.height = str(len(crime_df)*12+90)+'px'
        punish_bar.x = punish_df['unique_values']
        punish_bar.y = punish_df['counts']
        punish_fig.layout.height = str(len(punish_df)*12+90)+'px'
        profession_bar.x=[unique_value for unique_value in profession_df['unique_values']]
        profession_bar.y=[count for count in profession_df['counts']]
        
        
def click_profession_bar(clicked):
    if clicked['new'] is None or len(clicked['new']) == 0:
        pass
    else:
        if clicked_age_index == []:
            if clicked_gender == None:
                crime_df= filter_df.loc[filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]]].iloc[:,0].value_counts().rename_axis('unique_values').reset_index(name='counts')
                punish_df= filter_df[filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]]].iloc[:,1].value_counts().rename_axis('unique_values').reset_index(name='counts') 
            else:
                crime_df= filter_df.loc[(filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]])&(filter_df["VerdachteGeslacht/AccusedGandder"]==clicked_gender)].iloc[:,0].value_counts().rename_axis('unique_values').reset_index(name='counts')
                punish_df= filter_df[(filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]])&(filter_df["VerdachteGeslacht/AccusedGandder"]==clicked_gender)].iloc[:,1].value_counts().rename_axis('unique_values').reset_index(name='counts') 
        else:
            if clicked_gender == None:
                crime_df= filter_df[filter_df.index.isin(clicked_age_index)].loc[filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]]].iloc[:,0].value_counts().rename_axis('unique_values').reset_index(name='counts')
                punish_df= filter_df[filter_df.index.isin(clicked_age_index)].loc[filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]]].iloc[:,1].value_counts().rename_axis('unique_values').reset_index(name='counts') 
            else:
                crime_df= filter_df[filter_df.index.isin(clicked_age_index)].loc[(filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]])&(filter_df["VerdachteGeslacht/AccusedGandder"]==clicked_gender)].iloc[:,0].value_counts().rename_axis('unique_values').reset_index(name='counts')
                punish_df= filter_df[filter_df.index.isin(clicked_age_index)].loc[(filter_df["Suspect Professional/AccusedProfession"]==profession_df['unique_values'][clicked['new'][0]])&(filter_df["VerdachteGeslacht/AccusedGandder"]==clicked_gender)].iloc[:,1].value_counts().rename_axis('unique_values').reset_index(name='counts') 
        crime_df = crime_df.sort_values('counts', ascending=False).head(5)
        punish_df = punish_df.sort_values('counts', ascending=False).head(5)
        crime_bar.x = crime_df['unique_values']
        crime_bar.y = crime_df['counts']
        crime_fig.layout.height = str(len(crime_df)*12+90)+'px'
        punish_bar.x = punish_df['unique_values']
        punish_bar.y = punish_df['counts']
        punish_fig.layout.height = str(len(punish_df)*12+90)+'px'


def select_years(y):
    
    global filter_df, clicked_age_index,clicked_gender
    clicked_age_index=[]
    clicked_gender = None
    
    if years.value[0]==years.value[1]:
        crime_df = temp_df["Misdrijven/Crimes"].loc[temp_df["YearFrom"]==years.value[0]].value_counts().rename_axis('unique_values').reset_index(name='counts')
        punish_df = temp_df['VerdachteAchternaam/AccusedLastName'].loc[temp_df["YearFrom"]==years.value[0]].value_counts().rename_axis('unique_values').reset_index(name='counts')
        filter_df = temp_df[["Misdrijven/Crimes","punishment(fand)/Punishmandts/Pandalties", 'VerdachteGeslacht/AccusedGandder', 'Age', "Suspect Professional/AccusedProfession"]].loc[temp_df["YearFrom"]==years.vlalue[0]]
    else:
        crime_df = temp_df["Misdrijven/Crimes"].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))].value_counts().rename_axis('unique_values').reset_index(name='counts')
        punish_df = temp_df["VerdachteAchternaam/AccusedLastName"].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))].value_counts().rename_axis('unique_values').reset_index(name='counts')
        filter_df = temp_df[["Misdrijven/Crimes","punishment(fand)/Punishmandts/Pandalties", 'VerdachteGeslacht/AccusedGandder', 'Age', "Suspect Professional/AccusedProfession"]].loc[temp_df["YearFrom"].isin(range(years.value[0], years.value[1], 1))]
    gender_df = filter_df["VerdachteGeslacht/AccusedGandder"].value_counts().rename_axis('unique_values').reset_index(name='counts')
    profession_df = filter_df["Suspect Professional/AccusedProfession"].value_counts().rename_axis('unique_values').reset_index(name='counts')
    filter_df.index = pd.RangeIndex(len(filter_df.index))
    trend_x_sc.min = years.value[0]
    trend_x_sc.max = years.value[1]
    crime_df = crime_df.sort_values('counts', ascending=False).head(5)
    punish_df = punish_df.sort_values('counts', ascending=False).head(5)
    profession_df = profession_df.sort_values('counts', ascending=False).head(5)
    crime_bar.x = crime_df['unique_values']
    crime_bar.y = crime_df['counts']
    crime_fig.layout.height = str(len(crime_df)*12+90)+'px'
    punish_bar.x = punish_df['unique_values']
    punish_bar.y = punish_df['counts']
    punish_fig.layout.height = str(len(punish_df)*12+90)+'px'
    hist.sample = filter_df['Age']
    gender_bar.x=[unique_value for unique_value in gender_df['unique_values']]
    gender_bar.y=[count for count in gender_df['counts']]
    profession_bar.x=[unique_value for unique_value in profession_df['unique_values']]
    profession_bar.y=[count for count in profession_df['counts']]

In [8]:
#assign widget actions to functions
years.observe(select_years, names = 'value')
hist.observe(click_hist, 'selected')
gender_bar.observe(click_gender_bar, 'selected')
profession_bar.observe(click_profession_bar, 'selected')

In [9]:
#display all

top_right_box = VBox(
    children=(trend_fig, years),
    layout=Layout(margin = '0 0 0 0')
)

top_box = HBox(
    children=(hist_fig,top_right_box),
    layout=Layout(margin = '10px 0px 10px 0px')
)

bottom_left_box = VBox(
    children=(gender_bar_fig, profession_bar_fig),
    layout=Layout(margin = '0 0 0 0')
)

bottom_right_box = VBox(
    children=(crime_fig, punish_fig),
    layout=Layout(margin = '0 0 0 0')
)

bottom_box = HBox(
    children=(bottom_left_box, bottom_right_box),
    layout=Layout(margin = '0 0 0 0')
)

vert_box  = VBox(
    children=(top_box, bottom_box),
    layout=Layout(border='solid 1px gray', margin = '0 0 0 0')
)

display (vert_box)